In [ ]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import os,cv2

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Add, MaxPooling2D, AveragePooling2D, Dense
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D, Concatenate, Flatten
from tensorflow.keras.activations import relu, softmax, sigmoid, tanh
# from tensorflow.keras import initializers

from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
from keras.layers import Input

In [ ]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
# from keras.activations import sigmoid
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!git clone https://github.com/Sunport15/SCZDetection.git

fatal: destination path 'SCZDetection' already exists and is not an empty directory.


In [ ]:
data_path='/content/SCZDetection/ReChannelImagesWithOverlappedWindows'
data_dir_list = ['alpha', 'beta', 'theta']
img_data_list_alpha=[]
img_data_list_beta=[]
img_data_list_theta=[]
labels = []

img_cols = 64
img_rows = 64

for dataset in data_dir_list:
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for lab in os.listdir(data_path+'/'+ dataset):
        for img in os.listdir(data_path+'/'+ dataset+'/'+lab):
            input_img=cv2.imread(data_path+'/'+ dataset+'/'+lab+ '/'+ img)
    #         input_img=cv2.IMREAD_GRAYSCALE

            input_img_resize=cv2.resize(input_img,(img_cols, img_rows))
            if dataset == 'alpha':
                img_data_list_alpha.append(input_img_resize)
                labels.append(lab)
            elif dataset == 'beta':
                img_data_list_beta.append(input_img_resize)
            elif dataset == 'theta':
                img_data_list_theta.append(input_img_resize)
    #     print(np.array(img_data_list).shape)

label=np.array(labels)
data_alpha = np.array(img_data_list_alpha)
data_beta = np.array(img_data_list_beta)
data_theta = np.array(img_data_list_theta)
data_alpha = data_alpha.astype('float32')
data_alpha = data_alpha/255

data_beta = data_beta.astype('float32')
data_beta = data_beta/255

data_theta = data_theta.astype('float32')
data_theta = data_theta/255

data_alpha.shape
data_beta.shape
data_theta.shape

Loaded the images of dataset-alpha

Loaded the images of dataset-beta

Loaded the images of dataset-theta



(7392, 64, 64, 3)

In [ ]:
num_classes = 2
label = np_utils.to_categorical(label, num_classes)

In [ ]:
print(data_alpha.shape)
print(data_beta.shape)
print(data_theta.shape)
print(label.shape)

(7392, 64, 64, 3)
(7392, 64, 64, 3)
(7392, 64, 64, 3)
(7392, 2)


In [ ]:
data_alpha = data_alpha[:2800]
data_beta = data_beta[:2800]
data_theta = data_theta[:2800]
label = label[:2800]

In [ ]:
a_train, a_test, b_train, b_test, th_train, th_test, y_train, y_test = train_test_split(data_alpha,data_beta,data_theta, label, test_size=0.05, random_state=2)
print(a_train.shape)
print(b_train.shape)
print(th_train.shape)
print(y_train.shape)
print(a_test.shape)
print(b_test.shape)
print(th_test.shape)
print(y_test.shape)

(2660, 64, 64, 3)
(2660, 64, 64, 3)
(2660, 64, 64, 3)
(2660, 2)
(140, 64, 64, 3)
(140, 64, 64, 3)
(140, 64, 64, 3)
(140, 2)


In [ ]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, Add, BatchNormalization,Conv1D
import tensorflow as tf
from keras.models import Model

def cbam_block(cbam_feature, ratio=8):
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    H, W, C = map(int, input_feature.get_shape()[1:])
    channel = C

    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1,1,channel))(avg_pool)
    #assert avg_pool._keras_shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    #assert avg_pool._keras_shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    #assert avg_pool._keras_shape[1:] == (1,1,channel)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    #assert max_pool._keras_shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    #assert max_pool._keras_shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    #assert max_pool._keras_shape[1:] == (1,1,channel)

    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    H, W, C = map(int, input_feature.get_shape()[1:])
    if K.image_data_format() == "channels_first":
        channel = C
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = C
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    #assert avg_pool._keras_shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    #assert max_pool._keras_shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    #assert concat._keras_shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)
    #assert cbam_feature._keras_shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

In [ ]:
input1 = Input(shape=(img_cols, img_rows, 3))
input2 = Input(shape=(img_cols, img_rows, 3))
input3 = Input(shape=(img_cols, img_rows, 3))

################## alpha band #########################################
c1 = Conv2D(filters=32, kernel_size=(3,3), strides=1, activation=relu)(input1)

# Block 1
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(c1)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb1 = cbam_block(c1,4)
# print(c1.shape)
# print(cb1.shape)
x_c = Add()([c1, cb1])

# Block 2
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb2 = cbam_block(c1,4)
x_c = Add()([c1, cb2])

# Block 3
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(7,7), strides=1, activation=relu)(c1)
cb3 = cbam_block(c1,4)
x_c = Add()([c1, cb3])
x_a = Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), activation=relu)(x_c)
x_a = GlobalAveragePooling2D()(x_a)

################ beta band #####################################
c1 = Conv2D(filters=32, kernel_size=(3,3), strides=1, activation=relu)(input2)

# Block 1
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(c1)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb1 = cbam_block(c1,4)
# print(c1.shape)
# print(cb1.shape)
x_c = Add()([c1, cb1])

# Block 2
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb2 = cbam_block(c1,4)
x_c = Add()([c1, cb2])

# Block 3
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(7,7), strides=1, activation=relu)(c1)
cb3 = cbam_block(c1,4)
x_c = Add()([c1, cb3])
x_b = Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), activation=relu)(x_c)

x_b = GlobalAveragePooling2D()(x_b)

################ theta band ################################
c1 = Conv2D(filters=32, kernel_size=(3,3), strides=1, activation=relu)(input3)

# Block 1
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(c1)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb1 = cbam_block(c1,4)
# print(c1.shape)
# print(cb1.shape)
x_c = Add()([c1, cb1])

# Block 2
c1 = Conv2D(filters=64, kernel_size=(3,3), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(c1)
cb2 = cbam_block(c1,4)
x_c = Add()([c1, cb2])

# Block 3
c1 = Conv2D(filters=64, kernel_size=(5,5), strides=1, activation=relu)(x_c)
c1 = Conv2D(filters=64, kernel_size=(7,7), strides=1, activation=relu)(c1)
cb3 = cbam_block(c1,4)
x_c = Add()([c1, cb3])
x_th = Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), activation=relu)(x_c)

x_th = GlobalAveragePooling2D()(x_th)

########## merge all bands ####################

m1 = Concatenate()([x_a,x_b,x_th])
# m1 = Conv2D(filters=16, kernel_size=(1,1), strides=(1, 1), activation=relu)(m1)
# m1 = GlobalAveragePooling2D()(m1)

m1 = Dense(units=256, activation=relu)(m1)
m1 = Dense(units=128, activation=relu)(m1)

out_c = Dense(units=2, activation=sigmoid)(m1)

model = Model([input1,input2,input3], out_c)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 62, 62, 32)   896         ['input_1[0][0]']                
                                                                                              

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(
                              patience=30,
                              min_delta=0.001,
                              monitor="val_accuracy",
                              restore_best_weights=True
                              )
# Define the model checkpoint callback to save the best weights
checkpoint = ModelCheckpoint('/workspace/Sunidhi/Conference_paper/checkpoint/'+'-{epoch:02d}.h5', monitor='val_accuracy', save_best_only=True)


In [ ]:
from keras import optimizers as optim
from keras import backend as K
from keras import optimizers

model.compile(loss='categorical_crossentropy',optimizer = keras.optimizers.Nadam(learning_rate=0.0001),metrics=['accuracy'])
# Train the model
history = model.fit([a_train, b_train,th_train], y_train, epochs=100, batch_size=8, validation_data=([a_test,b_test,th_test], y_test),callbacks=[early_stopping, checkpoint], verbose=1)


Epoch 1/100
  9/333 [..............................] - ETA: 31:04 - loss: 0.6269 - accuracy: 0.8889

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(train_acc))
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'g', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.grid()
plt.legend()
plt.show()

plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Get the ground truth
ground_truth = np.argmax(y_test, axis=1)

# Get the predictions from the model
predictions = model.predict([a_test,b_test,th_test])
predicted_classes = np.argmax(predictions, axis=1)

errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors), y_test.shape[0]))


In [ ]:
accuracy = (len(y_test) - len(errors)) / len(y_test) * 100
print("Accuracy: {:.2f}%".format(accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
cm = confusion_matrix(y_true=ground_truth, y_pred=predicted_classes)
print(cm)
cm = np.array(cm)
# Normalise
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(5,4))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=np.unique(labels), yticklabels=np.unique(labels), cmap="YlGnBu")
plt.ylabel('Actual', fontsize=15)
plt.xlabel('Predicted', fontsize=15)
plt.show(block=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ground_truth, predicted_classes, target_names=np.unique(labels)))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
y_pred_proba = model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test[:, 1], y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()